In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from constants import dataset_metadata, verbose
from apriori import apriori_for_dataset

In [3]:
dataset_name = 'groceries'

In [4]:
%%time
apriori_for_dataset(dataset_name, dataset_metadata)

Apriori on groceries with minimum support threshold value(s) from [70, 150, 200]

min supp: 70 	pass_1(1 chunks):	 31 (20.0sec) 	pass_2(2 chunks):	 72  (26.9sec) 	merged:	 103
min supp: 150 	pass_1(1 chunks):	 36 (5.8sec) 	pass_2(2 chunks):	 29  (7.9sec) 	merged:	 65
min supp: 200 	pass_1(1 chunks):	 9 (4.3sec) 	pass_2(2 chunks):	 36  (5.5sec) 	merged:	 45

CPU times: user 40.1 s, sys: 67.3 ms, total: 40.2 s
Wall time: 40.3 s
